# OSP model

Changes:
- Absolute path
- Flexible zero-error-radius
- Examine output by teaching signal and slots

## Import libraries

In [ ]:
%load_ext lab_black

import h5py, pickle, os
import tensorflow as tf
import numpy as np
import pandas as pd

import meta, data_wrangling, modeling, evaluate
from IPython.display import clear_output

## Parameters block for Papermill
- Instead of using model_cfg directly, this extra step is needed for batch run using Papermill
- Consider carefully the variable type in each cfg setting (Probably automatically check it later...)
    - Do not use integer (e.g., 1, 2, 3, 0) in variables that can be float32 (e.g., w_oh_noise, tau...)
    - Use integer with a dot instead (e.g., 1., 2., 3., 0.)
- To use attractor, two params must be config, 
    - 1) embed_attractor_cfg --> json cfg file of the pretrain attractor 
    - 2) embed_attractor_h5 --> h5 file of the exact weight (e.g. ep0500.h5 #epoch or c90.h5 #correct rate)

In [ ]:
code_name = "booo2"

sample_name = "jay"
rng_seed = 53797
use_semantic = False

# Model architechture
input_dim = 4
output_dim = 4
hidden_units = 100
cleanup_units = 20

pretrain_attractor = False  # Load pretrained or not

# embed_attractor_cfg = 'models/Attractor_{0:02d}/model_config.json'.format(
#     cleanup_units
# )
# embed_attractor_h5 = 'c00.h5'

rnn_activation = "sigmoid"
regularizer_const = None
w_initializer = 0.1  # range of uniform random
zero_error_radius = 0.5  # When True, zer value = 0.1, hardcoded in modeling.zer_bce()


p_noise = 0.0
tau = 1 / 3
max_unit_time = 4.0
output_ticks = 2

# Training
optimizer = "adam"
n_mil_sample = 0.1
batch_size = 32
learning_rate = 0.005
save_freq = 10

bq_dataset = None
batch_unique_setting_string = None

## Construct model configuration

In [ ]:
d = {}
for v in meta.model_cfg.minimal_cfgs:
    d[v] = globals()[v]

for v in meta.model_cfg.aux_cfgs:
    try:
        d[v] = globals()[v]
    except:
        pass

cfg = meta.model_cfg(**d)

tf.random.set_seed(cfg.rng_seed)
data = data_wrangling.my_data(cfg)

### Replace data module by toy data
- Input set to 1, 0, 0, 1
- Output is 0, 1, 1, 0

In [ ]:
toy_input = np.random.choice([0, 1], size=[128, 4])
data.x_train = toy_input.astype("float32")
not_x = 1 - toy_input
data.y_train = not_x.astype("float32")
data.sample_p = np.tile(1, 128) / 128

# Modeling

### Custom metrics (in development)

In [ ]:
class Out0(tf.keras.metrics.Metric):
    """Export last slot average output in last batch of a epoch
    """

    def __init__(self, name="output0", **kwargs):
        super(Out0, self).__init__(name=name, **kwargs)
        self.out = self.add_weight(name="out0", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.out.assign(tf.reduce_mean(y_pred[y_true == 0]))

    def result(self):
        return self.out

    def reset_states(self):
        self.out.assign(0.0)


class Out1(tf.keras.metrics.Metric):
    """Export last slot average output in last batch of a epoch
    """

    def __init__(self, name="output1", **kwargs):
        super(Out1, self).__init__(name=name, **kwargs)
        self.out = self.add_weight(name="out1", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.out.assign(tf.reduce_mean(y_pred[y_true == 1]))

    def result(self):
        return self.out

    def reset_states(self):
        self.out.assign(0.0)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Input, concatenate, multiply, RepeatVector
from tensorflow.keras.optimizers import Adam, SGD


def build_model(cfg, training=True):

    # Select training or testing mode
    cfg.noise_on() if training else cfg.noise_off()

    input_o = Input(shape=(cfg.input_dim,), name="Input_O")
    input_o_t = RepeatVector(cfg.n_timesteps, name="Input_Ot")(input_o)

    rnn_model = modeling.rnn(cfg)(input_o_t)
    model = Model(input_o, rnn_model)

    op = Adam(learning_rate=cfg.learning_rate, beta_1=0.0, beta_2=0.999, amsgrad=False)
    me = ["BinaryAccuracy", "mse", Out0(), Out1()]

    if cfg.zero_error_radius is not None:
        print(f"Using zero-error-radius of {cfg.zero_error_radius}")

        model.compile(
            loss=modeling.CustomBCE(radius=cfg.zero_error_radius),
            optimizer=op,
            metrics=me,
        )

    elif cfg.zero_error_radius is None:
        print(f"No zero-error-radius")
        model.compile(loss="binary_crossentropy", optimizer=op, metrics=me)

    model.summary()
    return model


model = build_model(cfg)

## Training

In [ ]:
tboard = tf.keras.callbacks.TensorBoard(
    log_dir=f"batch_log/{cfg.code_name}", histogram_freq=1
)


model.fit(
    data_wrangling.sample_generator(cfg, data),
    steps_per_epoch=1,
    epochs=100,
    verbose=1,
    callbacks=[tboard],
)

In [ ]:
# !tensorboard --logdir $cfg.path_log_folder